## NLP
### Bag-of-Words, Word2Vec e Embeddings
<b>Objetivo: </b> Entender como transformar texto em representações vetoriais numéricas que alimentarão um modelo.
<br><b>Autora:</b> Renata Gotler

Vamos continuar usando os dados de comentários da americanas que exploramos na aula passada.<br>

In [73]:
import nltk
import spacy
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec, Phrases

from src.preprocessing import TextPreprocessing

prep = TextPreprocessing()

In [74]:
nltk.download("punkt")
nltk.download("rslp")
nltk.download("stopwords")

lemmatizer = spacy.load('pt_core_news_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
url = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/main/B2W-Reviews01.csv'

df_reviews = pd.read_csv(url)
df_reviews = df_reviews[["review_text", "overall_rating"]].dropna()

/tmp/ipykernel_939/1634640095.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reviews = pd.read_csv(url)


#### Bag-of-Words
Representa cada documento como um vetor de frequências de palavras. Esse vetor será de números inteiros, visto que é uma contagem de palavras, e terá tamanho fixo, o tamanho do vocabulário passado.

In [77]:
vectorizer_full = CountVectorizer()
X_raw = vectorizer_full.fit_transform(df_reviews["review_text"])

O X será terá formato: número de registros x tamanho do vocabulário. Na aula passada usamos esses mesmos dados e sem limpeza, obtivemos um vocabulário de cerca de 75k tokens, aqui temos cerca de 50k, isso acontece porque o CountVectorizer por padrão já transforma o texto para minúsculo e já aplica algumas limpezas.

In [78]:
X_raw.shape

(129098, 50336)

Abaixo podemos ver que alguns tokens são formados somente por números e podem prejudicar e deixar nosso modelo mais complexo, tendo ainda espaço para melhorias. Vamos aplicar algumas limpezas da aula passada para melhorar esses vetores?

In [79]:
print(vectorizer_full.get_feature_names_out()[:5])

['00' '000' '0000' '00000' '000000']


In [81]:
df_reviews["review_text_cleaned"] = df_reviews["review_text"].apply(lambda x: prep.preprocess_text(
    text=x,
    apply_lower=True,
    remove_ponctuation=True,
    remove_numbers=True,
    clean_html=True,
    apply_unidecode=True,
    remove_stopwords=True,
    remove_short_tokens=True,
    min_tokens_size=2,
    limit_consecutive_chars=True,
    max_consecutive_char=2,
    apply_lemmitization=True,
    lemmatizer=lemmatizer))

In [82]:
X_cleaned = vectorizer_full.fit_transform(df_reviews["review_text_cleaned"])
X_cleaned.shape

(129098, 37539)

Ainda temos algumas oportunidades de melhoria, mas por hora vamos seguir, 37539 já é bem melhor do que 50336.

In [83]:
print(vectorizer_full.get_feature_names_out()[:5])

['1a' '2e' 'aa' 'aabar'
 'aabbgdzfhijvfgjkbghjhgfvfefvhycfjufvgjchhfdfbnkoufdcmkjtexnjg']


Abaixo podemos ver que é bem difícil limpar um texto 100%, dessa forma, o ideal é ir testando e vendo o impacto até chegar no resultado desejado. Por exemplo, abaixo vemos alguns erros de digitação, como o `Aabei` deveria ser `Acabei`, que poderiamos corrigir com as correspondências fuzzy. 

In [87]:
df_reviews[df_reviews["review_text_cleaned"].str.contains("aabar")]["review_text"].iloc[0]

'Aabei de instalar.Muito facil e rapido de ser instalada.Sua imagem e otima tanto durante o dia quanto a noite.Eu recomendo.Excelente produto cumpre o que diz perfeitamente.'

In [88]:
df_reviews[df_reviews["review_text_cleaned"].str.contains("aabar")]["review_text_cleaned"].iloc[0]

'aabar instalar facil rapir instalar imagem otimo tanto durante dia quanto noite recomer excelente produto cumprir dizer perfeitamente'

Para fins didáticos, vou reduzir a quantidade de dados para poder mostrar como é a matriz resultante.

In [89]:
vectorizer_small = CountVectorizer()
X_small = vectorizer_small.fit_transform(df_reviews["review_text_cleaned"][:3])
X_small.shape

(3, 37)

Teremos o vetor registro x tamanho de vocabulário. Abaixo podemos ver que o CountVectorizer somente conta a frequência de quantas vezes um token apareceu naquele texto.

In [90]:
vectorizer_small.get_feature_names_out()

array(['acrilico', 'agilidade', 'americano', 'apenas', 'arroz', 'compra',
       'comprar', 'conseguir', 'consumidor', 'contente', 'copo',
       'costumar', 'cozimento', 'devolucao', 'eletrica', 'em', 'entregar',
       'esperar', 'esse', 'exatamente', 'japonês', 'levar', 'lir',
       'minuto', 'outro', 'panela', 'praticidade', 'problema', 'produto',
       'raper', 'rapir', 'recomer', 'superar', 'tempo', 'trocar', 'unico',
       'usar'], dtype=object)

In [91]:
X_small.toarray()

array([[0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 1, 1, 0],
       [1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
        0, 2, 2, 3, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1]])

Por exemplo, podemos ver que temos um 3 no terceiro registro. Vamos ver se confere?
<br> Para isso, primeiro vamos encontrar o token do terceiro registro que aparece três vezes.

In [93]:
token_index = list(X_small.toarray()[2]).index(3)
vectorizer_small.get_feature_names_out()[token_index]

'panela'

Realmente, `panela` aparece três vezes no terceiro registro! Viu? CountVectorizer é simplesmente uma matrix que conta quantos tokens aparecem em cada registro, onde cada token vira uma coluna e cada registro, uma linha.

In [94]:
df_reviews["review_text_cleaned"].iloc[2]

'superar agilidade praticidade outro panela eletrica costumar usar outro panela cozimento arroz japonês levar tempo minuto em esse panela rapir exatamente minuto recomer'

A limpeza dos textos é bem difícil de ser feita 100%, podemos ver abaixo que já fizemos um excelente trabalho, então vamos ver os resultados que temos com a limpeza feita até agora, se o resultado atingido não for tão bom quanto o que queremos, iteramos para limpar mais. Isso é lidar com dados da vida real.

#### Bag of n-gram

Um parâmetro bem interessante que podemos explorar é o `ngram_range`, criando uma bag of n-grams. Esse parâmetro ajuda na geração de contexto, criando um conjunto de tokens.

In [95]:
vectorizer_n_gram = CountVectorizer(ngram_range=(1, 2))
X_n_gram = vectorizer_n_gram.fit_transform(df_reviews["review_text_cleaned"][:3])
X_n_gram.shape

(3, 75)

Primeiramente, veja que de um vocabulário de 37 tokens, agora com as combinações chegamos a 75. Dessa forma, temos que usar o n-gram com cuidado, visto que ele aumenta bastante a cardinalidade.

In [96]:
vectorizer_n_gram.get_feature_names_out()

array(['acrilico', 'agilidade', 'agilidade praticidade', 'americano',
       'americano trocar', 'apenas', 'apenas conseguir', 'arroz',
       'arroz japonês', 'compra', 'compra entregar', 'comprar',
       'comprar lir', 'conseguir', 'conseguir comprar', 'consumidor',
       'consumidor problema', 'contente', 'contente compra', 'copo',
       'copo acrilico', 'costumar', 'costumar usar', 'cozimento',
       'cozimento arroz', 'devolucao', 'devolucao produto', 'eletrica',
       'eletrica costumar', 'em', 'em esse', 'entregar', 'entregar raper',
       'esperar', 'esse', 'esse panela', 'exatamente',
       'exatamente minuto', 'japonês', 'japonês levar', 'levar',
       'levar tempo', 'lir', 'lir copo', 'minuto', 'minuto em',
       'minuto recomer', 'outro', 'outro panela', 'panela',
       'panela cozimento', 'panela eletrica', 'panela rapir',
       'praticidade', 'praticidade outro', 'problema',
       'problema americano', 'problema esperar', 'produto',
       'produto consumidor'

#### TF-IDF

Pondera as palavras com base na frequência no documento e na inversa da frequência nos documentos. Dessa forma, esse algoritmo ajuda a reduzir o peso de palavras comuns e aumentar o peso de palavras distintivas.

O TF-IDF tem as mesmas dimensões que o CountVectorizer, o que vão mudar são os números na matrix. Ao invés de uma contagem, temos uma ponderação de acordo com a frequência do token dentro do registro e da frequência do token nos demais registros.

In [97]:
tf_idf = TfidfVectorizer(norm="l1", smooth_idf=False)
X_tf_idf = tf_idf.fit_transform(df_reviews["review_text_cleaned"])
X_tf_idf.shape

(129098, 37539)

In [98]:
set(tf_idf.get_feature_names_out() == vectorizer_full.get_feature_names_out())

{True}

Vamos ver mais a fundo como funciona o TF-IDF? Começando pelo TF.

In [109]:
tf = TfidfVectorizer(norm="l1", use_idf=False)
X_tf = tf.fit_transform(df_reviews["review_text_cleaned"][:3])
X_tf.toarray()

array([[0.        , 0.        , 0.07692308, 0.        , 0.        ,
        0.07692308, 0.        , 0.        , 0.07692308, 0.07692308,
        0.        , 0.        , 0.        , 0.07692308, 0.        ,
        0.        , 0.07692308, 0.07692308, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.15384615, 0.07692308, 0.07692308,
        0.        , 0.        , 0.        , 0.        , 0.07692308,
        0.07692308, 0.        ],
       [0.16666667, 0.        , 0.        , 0.16666667, 0.        ,
        0.        , 0.16666667, 0.16666667, 0.        , 0.        ,
        0.16666667, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.16666667, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.     

Já vimos que o token `panela` se repete 3 vezes no terceiro registro, por isso, ele possui um valor (0.13043478) que é o triplo dos demais valores (0.043478260869565216), com exceção do token `minutos`, que aparece duas vezes, com valor 0.08695652, enquanto todos os demais ficaram com 0.04347826 (equivalente a aparecer uma vez no registro). Isso porque no TF, temos uma normalização da quantidade de vezes que um token aparece em um registro pela quantidade total de tokens do registro.

In [110]:
1 / len(df_reviews["review_text_cleaned"].iloc[2].split(" "))

0.043478260869565216

Dessa forma, todas as linhas somam 1.

In [ ]:
X_tf[0].sum()

0.9999999999999998

Temos 3 registros nesse treinamento, então o IDF será calculado como log(3/quantidade de registros que o token apareceu)+1. Como vemos, os tokens apareceram unicamente em seus registros, isso aconteceu muito porque limpamos bastante o texto, retirando stopwords ,por exemplo, que se repetiram mais entre os registros, com isso todos os tokens tiveram o valor log(3/1) + 1 = 2.09861229.
<br> Percebam também que o IDF será uma matrix do tamanho da quantidade do vocabulário (quantidade de variáveis).

In [112]:
tf_idf_small = TfidfVectorizer(norm="l1", smooth_idf=False)
X_tf_idf_small = tf_idf_small.fit_transform(df_reviews["review_text_cleaned"][:3])
tf_idf_small.idf_

array([2.09861229, 2.09861229, 2.09861229, 2.09861229, 2.09861229,
       2.09861229, 2.09861229, 2.09861229, 2.09861229, 2.09861229,
       2.09861229, 2.09861229, 2.09861229, 2.09861229, 2.09861229,
       2.09861229, 2.09861229, 2.09861229, 2.09861229, 2.09861229,
       2.09861229, 2.09861229, 2.09861229, 2.09861229, 2.09861229,
       2.09861229, 2.09861229, 2.09861229, 2.09861229, 2.09861229,
       2.09861229, 2.09861229, 2.09861229, 2.09861229, 2.09861229,
       2.09861229, 2.09861229])

In [115]:
len(tf_idf_small.idf_), X_tf_idf_small.shape

(37, (3, 37))

Por fim, podemos multiplicar nossas matrixes (TF x IDF) temos o TF-IDF. O TfidfVectorizer ainda aplica uma normalização final, para que a soma dos valores de cada registro resulte em 1, o que acabou sendo igual ao TF porque nosso IDF ficou igual para todos os tokens.

In [116]:
X_tf_idf_small.toarray()

array([[0.        , 0.        , 0.07692308, 0.        , 0.        ,
        0.07692308, 0.        , 0.        , 0.07692308, 0.07692308,
        0.        , 0.        , 0.        , 0.07692308, 0.        ,
        0.        , 0.07692308, 0.07692308, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.15384615, 0.07692308, 0.07692308,
        0.        , 0.        , 0.        , 0.        , 0.07692308,
        0.07692308, 0.        ],
       [0.16666667, 0.        , 0.        , 0.16666667, 0.        ,
        0.        , 0.16666667, 0.16666667, 0.        , 0.        ,
        0.16666667, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.16666667, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.     

O TF-IDF é bem interessante por penalisar tokens que se repetem muito entre registros, assumindo que se um token aparece frequentemente, ele não é um token importante para ajudar a distinguir entre os registros. Imagine um classificador de spam, as sentenças: "Assinado", "Fico à disposição", "Abraços", se repetem em vários emails e não nos ajudam a identificar um spam, certo? Já as sentenças: "Perca peso", "Cem por cento garantido", já não se repetem tanto nos emails e nos ajudam mais na classificação.
<br> Assim como o CountVectorizer, o TfidfVectorizer também possui parâmetros bem interessantes como o `ngram_range` que funciona da mesma forma e nos dá mais contexto ao custo de uma maior cardinalidade.

#### Word2Vec

Família de arquitetura de modelos e otimizações para criação de embeddings que cria vetores de palavras com base em contextos locais. Dessa forma, vetores distantes possuem significados diferentes e vetores próximos representam palavras com significados similares. Por exemplo, as palavras “mulher” e “rainha” estão perto no espaço vetorial, enquanto “mulher” e “muralha” estão mais distantes.

Abaixo vamos usar biblioteca gensim para testar o modelo padrão, que é o CBoW.


In [126]:
sentences = df_reviews["review_text_cleaned"].str.split(" ").to_list()
model = Word2Vec(sentences, vector_size=10, window=2, min_count=1, workers=4)

Perceba que o vetor tem tamanho definido pelo parâmetro vector_size, no caso, 10. Em geral, você deve fazer otimizações de hiperâmetros, como o vector_size, window_size e os demais para entender o melhor modelo para o seu caso.
<br>Por exemplo, um vector_size pequeno (50, 100) reduz a quantidade de recursos computacionais necessários e overfitting, indicado quando a quantidade de recursos é baixa ou você tem um dataset pequeno. Já maiores (200, 300) captura melhor as relações semânticas entre as palavras.

In [127]:
panela_vector = model.wv['panela']
panela_vector

array([ 0.0856761 ,  0.9885719 ,  1.3500437 , -0.815709  ,  1.0313131 ,
        0.970741  , -0.34947467,  1.6502775 , -2.5394058 , -1.1652255 ],
      dtype=float32)

Mesmo com somente um vetor de tamanho 10 e janela de 2, já conseguimos ver que a distância entre as palavras faz sentido, onde panela e prato fica mais perto do que panela e minutos.

In [152]:
model.wv.most_similar('panela', topn=1)

[('cama', 0.9742375612258911)]

In [137]:
model.wv.similarity('panela', 'prato')

0.81594384

In [140]:
model.wv.similarity('panela', 'minutos')

0.5390221

Para usar o skip-gram no lugar do CBoW, basta alterar o parâmetro `sg` para 1, conforme [documentação](https://radimrehurek.com/gensim/models/word2vec.html).

Se você quiser, pode salvar seu modelo para reutilizar posteriormente, ou retreiná-lo com novos dados.

In [136]:
model.save("../models/americanas_cbow.model")

In [141]:
model = Word2Vec.load("../models/americanas_cbow.model")
model.train([["hello", "world"]], total_examples=1, epochs=1)

(1, 2)

N-grams Embeddings

A biblioteca gensim possui uma classe chamada Phrases que automaticamente detecta frases comuns e cria tokens agrupando os tokens detectados com um demilitador _.

In [142]:
sentences = df_reviews["review_text_cleaned"].str.split(" ").to_list()
bigram_transformer = Phrases(sentences)
bigram_model = Word2Vec(bigram_transformer[sentences], min_count=1)

In [166]:
[key for key,_ in bigram_model.wv.key_to_index.items() if key not in model.wv.key_to_index.keys()][:5]

['antes_prazo', 'chegar_antes', 'super_rapir', 'custo_beneficio', 'ate_agora']

In [159]:
print(f"""Tamanho do vocabulário com n gram: {len(bigram_model.wv.key_to_index)}
Tamanho do vocabulário sem n gram: {len(model.wv.key_to_index)}""")

Tamanho do vocabulário com n gram: 42899
Tamanho do vocabulário sem n gram: 39012


Sempre precisamos analisar questões como bases enviesadas, mesmo em datasets onde não esperamos esse comportamento, como em comentário de produtos. Abaixo vemos um exemplo claro de viés.

In [150]:
bigram_model.wv.most_similar('mulher', topn=1)

[('familia', 0.9137730598449707)]

In [151]:
bigram_model.wv.most_similar('homem', topn=1)

[('desafio', 0.9510673880577087)]